In [1]:
import tabula
import numpy as np
import import_ipynb
import pandas as pd
import datetime as dt

In [2]:
def read_files(filepath,skiprows=0,sep=None,pages='all',relative_area=[],columns=None):
    if filepath.endswith('.sas7bdat') or filepath.endswith('.xpt'):
        try:
            data = pd.read_sas(filepath,encoding='ISO-8859-1')
        except:
            data = pd.read_sas(filepath,encoding='utf-8')
    if filepath.endswith('.csv'):
        try:
            data = pd.read_csv(filepath,sep=sep,engine='python')
        except:
            data = pd.read_csv(filepath,sep='|')
    if filepath.endswith('.TXT') or filepath.endswith('.txt'):
        data = pd.read_csv(filepath,sep='[\t|\|]',encoding='ISO-8859-1',engine='python')
    if filepath.endswith('.xlsx') or filepath.endswith('.xls') or filepath.endswith('.xlsm'):
        data = pd.read_excel(filepath,skiprows=skiprows)
    if filepath.endswith('.pdf'):
        data = tabula.read_pdf(filepath,pages=pages,relative_area=relative_area,columns=columns)
    return data

In [3]:
def lowercase_all_vars(df):
    df.columns = map(str.lower, df.columns)
    return df

In [4]:
def format_study_id(study_id):
    if len(study_id) >= 9:
        study_id = study_id.lower().replace('-','')
        study_id = study_id[:3] + '_' + study_id[3:6] + '_' + study_id[6:]
    return study_id

In [5]:
def standardize_datetime(x):
#     To be used if the original date is in the form ##-Mon-####
    try:
        date_dict = {'jan':'01','JAN':'01','feb':'02','FEB':'02','mar':'03','MAR':'03','apr':'04','APR':'04','may':'05','MAY':'05','jun':'06','JUN':'06','jul':'07','JUL':'07','aug':'08','AUG':'08','sep':'09','SEP':'09','oct':'10','OCT':'10','nov':'11','NOV':'11','dec':'12','DEC':'12'}
        if 'a' in x or 'e' in x or 'u' in x or 'i' in x or 'o' in x or 'c' in x or 'p' in x or 'A' in x or 'E' in x or 'U' in x or 'I' in x or 'O' in x or 'C' in x or 'P' in x:
            x = x.lower().split('-')
            x.reverse()
            x[1] = date_dict[x[1]]
            x = ''.join(x)
        if '-' in x:
            x = x.replace('-','')
        x = pd.to_datetime(x,format='%Y%m%d')
    except:
        pass
    return x

In [6]:
def convert_day_to_week(string):
    string1 = string
    if 'Day' in string1:
        if int(string[-2:])>0 and int(string[-2:])<8:
            string1 = 'Week 1'
        if int(string[-2:])>=8 and int(string[-2:])<15:
            string1 = 'Week 2'
        if int(string[-2:])>=15 and int(string[-2:])<22:
            string1 = 'Week 3'
        if int(string[-2:])>=22 and int(string[-2:])<29:
            string1 = 'Week 4'
        if int(string[-2:])>=29 and int(string[-2:])<36:
            string1 = 'Week 5'
        if int(string[-2:])>=36 and int(string[-2:])<43:
            string1 = 'Week 5'
        if int(string[-2:])>=43 and int(string[-2:])<50:
            string1 = 'Week 6'
        if int(string[-2:])>=50 and int(string[-2:])<57:
            string1 = 'Week 7'
    return string1

In [7]:
def multi_vals_col_to_list(vals):
    if pd.isnull(vals):
        vals = ''
        return vals
    vals = vals.split(',')
    return vals

In [8]:
def handle_nan(vals):
    if pd.isnull(vals):
        vals = ''
    return vals

In [9]:
def concat_patient_id(patient_id):
    if '-' in patient_id:
        patient_id = patient_id.replace('-','')
    return patient_id

In [10]:
def convert_age_to_dob(age,year_of_start):
    study_year = dt.date(year_of_start,1,1)
    dob = study_year - dt.timedelta(days=(age*365))
    return dob

In [11]:
def insert_row(df,idx,row_vals):
    upper = df[:idx]
    lower = df[idx:]
    upper.loc[idx] = row_vals
    new_df = pd.concat([upper,lower],axis=0).reset_index()
    new_df = new_df.drop(['index'],axis=1)
    return new_df

In [12]:
def remove_nan_cols(df):
    return df.dropna(axis=1,how='all')